In [9]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [10]:
%run __init__.ipynb

Successfully connected to MongoDB


In [11]:
hc = bob.HealthCheck()
hc_key = "99c0c879-0fe1-45c5-b5a6-95f0c2b1b203"
if use_healthchecks:
    step_1 = hc.start(hc_key)
    print(step_1)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-28


In [12]:
#--- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

Script execution completed at 28/04/2020 07:08:27. Time: --- 0.835174560546875 secnds ---
Dataframe FRANCE_INIT successfully save in database covid-19-dev in MongoDB. Time: --- 0.3419630527496338 secnds ---
Script execution completed at 28/04/2020 07:08:27. Time: --- 0.012402534484863281 secnds ---
Dataframe FRANCE_DB_CONCAT successfully save in database covid-19-dev in MongoDB. Time: --- 0.6412978172302246 secnds ---
Script execution completed at 28/04/2020 07:08:27. Time: --- 0.06134319305419922 secnds ---
Dataframe FRANCE_DB_CONSO successfully save in database covid-19-dev in MongoDB. Time: --- 0.5829553604125977 secnds ---
Script execution completed at 28/04/2020 07:08:27. Time: --- 0.28339362144470215 secnds ---
Dataframe FRANCE_DB_ALL successfully save in database covid-19-dev in MongoDB. Time: --- 13.096434116363525 secnds ---
Script execution completed at 28/04/2020 07:08:27. Time: --- 0.07615351676940918 secnds ---
Dataframe FRANCE_DB_TREND successfully save in database covid-

In [13]:
#--- Load data source
start_time = time.time()
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 28/04/2020 07:08:58. Time: --- 0.14926481246948242 secnds ---


In [14]:
#-- Covid-19 - DATA.GOUV.FR TENDANCES
start_time = time.time()
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    upper_filter.to_csv(output_folder + '105_UPPER_FILTER.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(upper_filter,'105_UPPER_FILTER',db_app,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain105

Dataframe 105 successfully save in database app-wsr-dev in MongoDB. Time: --- 16.389055967330933 secnds ---
Dataframe 105_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.02303314208984375 secnds ---
Script execution completed at 28/04/2020 07:09:15. Time: --- 17.164000272750854 secnds ---


,ENTITY,SCENARIO,GROUPS,GROUPS_ORDER,DATE_ORDER,BOTTOM_FILTER,UPPER_FILTER_M,UPPER_FILTER_R,LABEL_GROUPS,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,France,27/04/2020,Hospitalisations,1.0,20200427,Depuis le début,EVOLUTION,31 - Haute-Garonne,Occitanie,155.000000,cas,",.0f",28/04/2020 07:08:58
1,France,27/04/2020,Hospitalisations,1.0,20200427,Depuis le début,EVOLUTION,32 - Gers,Occitanie,41.000000,cas,",.0f",28/04/2020 07:08:58
2,France,27/04/2020,Hospitalisations,1.0,20200427,Depuis le début,EVOLUTION,33 - Gironde,Nouvelle-Aquitaine,253.000000,cas,",.0f",28/04/2020 07:08:58
3,France,27/04/2020,Hospitalisations,1.0,20200427,Depuis le début,EVOLUTION,34 - Hérault,Occitanie,123.000000,cas,",.0f",28/04/2020 07:08:58
4,France,27/04/2020,Hospitalisations,1.0,20200427,Depuis le début,EVOLUTION,35 - Ille-et-Vilaine,Bretagne,156.000000,cas,",.0f",28/04/2020 07:08:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,France,13/04/2020,Retours à domicile,3.0,20200413,15 derniers jours,VARIATION EN NB,France,France,532.000000,cas,"+,.0f",28/04/2020 07:08:58
67316,France,13/04/2020,Hospitalisations,1.0,20200413,15 derniers jours,VARIATION EN %,France,France,0.906363,%,"+,.2f",28/04/2020 07:08:58
67317,France,13/04/2020,Décès à l’hôpital,4.0,20200413,15 derniers jours,VARIATION EN %,France,France,3.620447,%,"+,.2f",28/04/2020 07:08:58
67318,France,13/04/2020,En réanimation,2.0,20200413,15 derniers jours,VARIATION EN %,France,France,-0.357462,%,"+,.2f",28/04/2020 07:08:58


In [15]:
#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
start_time = time.time()
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain107

Dataframe 107 successfully save in database app-wsr-dev in MongoDB. Time: --- 13.37002158164978 secnds ---
Script execution completed at 28/04/2020 07:09:29. Time: --- 13.538023471832275 secnds ---


,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,UNIT_VARP,PRECISION_VALUE,PRECISION_VAR,PRECISION_VARP,LAST_UPDATE
0,France,27/04/2020,20200427,Île-de-France,root,En réanimation,2.0,1859,-32,-1.692226,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
1,France,27/04/2020,20200427,Île-de-France,root,Hospitalisations,1.0,11488,-82,-0.708729,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
2,France,27/04/2020,20200427,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,155,5,3.333333,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
3,France,27/04/2020,20200427,32 - Gers,Occitanie,Hospitalisations,1.0,41,-1,-2.380952,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
4,France,27/04/2020,20200427,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,253,5,2.016129,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,France,19/03/2020,20200319,85 - Vendée,Pays de la Loire,Retours à domicile,3.0,5,5,inf,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
19196,France,19/03/2020,20200319,82 - Tarn-et-Garonne,Occitanie,Retours à domicile,3.0,0,0,NaN,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
19197,France,19/03/2020,20200319,81 - Tarn,Occitanie,Retours à domicile,3.0,2,0,0.000000,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15
19198,France,19/03/2020,20200319,80 - Somme,Hauts-de-France,Retours à domicile,3.0,12,3,33.333333,%,",.0f","+,.0f","+,.2f",28/04/2020 07:09:15


In [16]:
if use_healthchecks:
    step_2 = hc.done(hc_key)
    print(step_2)

End ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-28
